# Estimating Correlation Using Discrete Copulas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

from scipy.stats import truncnorm, norm, multivariate_normal, nbinom
import scipy.stats as stats

import tools as tl

# load data
data1_4 = pd.read_csv("mt-datasets/Donor1_CD4_Genes.csv")
# normalization
data_norm = tl.RPM(data1_4)
fitted_para = pd.read_csv('fitted_data/fitted_para.csv')


data_pro = data_norm[fitted_para.columns.to_list()]
data_pro = data_pro.reset_index(drop=True)
N = data_pro.shape[0]

To calculate the correlation between two datasets, the usual approach is to assume a bivariate normal distribution. However, this assumption may not always be appropriate. For our data, as observed in the previous section, the datasets themselves are not normally distributed. This implies that the bivariate normal model is not suitable, as it also assumes that each marginal distribution is normally distributed.
We introduce a new method called Copula. This approach allows us to leverage the simulated marginal distributions for each dataset that we have already obtained. 

## 1. Copula
To introduce Copula, we first need to present a key theorem known as Sklar's Theorem.

### 1.1 Introduction
- **Sklar's Theorem** states that any multivariate distribution can be represented in terms of its marginal distributions and a copula that captures the dependencies between them. Specifically:

  If $ F $ is a joint cumulative distribution function (CDF) with marginals $ F_1, F_2, \ldots, F_d $, then there exists a copula $ C$ such that

   $$
   F(x_1, x_2, \ldots, x_d) = C(F_1(x_1), F_2(x_2), \ldots, F_d(x_d))
   $$

   where $ F_i $ is the marginal CDF of the $ i $-th variable, and $ C $ is the **copula function** that describes the dependence structure between the variables.

Conversely, any copula $ C $ with marginal distributions $ F_1, F_2, \ldots, F_d $ can be used to construct a joint CDF $ F $ as given above.

This theorem allows us to separate the modeling of marginal distributions from the modeling of dependencies, providing flexibility in statistical modeling and analysis.



### 1.2 Gaussian Copula

In practice, a copula function can be chosen from a wide variety of families, with the Gaussian copula being one of the most popular options. Among all copulas, the Gaussian copula is particularly favored due to its simplicity and effectiveness in capturing linear dependencies. In this context, we will use the Gaussian copula to estimate correlations.

**Definition**: For the case where $ n = 2 $, the Gaussian copula models the dependence between two random variables $ X_1 $ and $ X_2 $. And the    Gaussian copula is defined as:
$$
C_\text{Gauss}(u_1, u_2; \rho) = \Phi_\rho(\Phi^{-1}(u_1), \Phi^{-1}(u_2)),
$$
where:

- $ u_1 = F_{X_1}(x_1) $ and $ u_2 = F_{X_2}(x_2) $ are the marginal distributions (CDFs) of $ X_1 $ and $ X_2 $, respectively.
- $ \Phi^{-1} $ is the inverse of the standard normal CDF.
- $ \Phi_\rho $ is the CDF of the bivariate normal distribution with correlation $ \rho $.

Here, we have copula function $C_\text{Gauss}(u_1, u_2; \rho)$, we can also write the copula density as:
$$
c_{\text{Gauss}}(u_1, u_2; \rho) = \frac{\exp\left( -\frac{1}{2(1 - \rho^2)} \left[ \left( \Phi^{-1}(u_1) \right)^2 - 2 \rho \Phi^{-1}(u_1) \Phi^{-1}(u_2) + \left( \Phi^{-1}(u_2) \right)^2 \right] \right)}{2 \pi \sqrt{1 - \rho^2}}
$$
Here, $\rho$ is the only parameter in Gaussian copula. To obtain the value of $\rho$, we will use Markov Chain Monte Carlo (MCMC) again.



In [ ]:
def gaussian_copula_cdf_vector(u1, u2, rho):
    """
    Compute the Gaussian copula CDF for two variables with vector inputs.

    Parameters:
    u1 (array-like): The first marginal CDF values (0 < u1 < 1).
    u2 (array-like): The second marginal CDF values (0 < u2 < 1).
    rho (float): The correlation coefficient between the two variables (-1 < rho < 1).

    Returns:
    array: The values of the Gaussian copula CDF.
    """

    u1 = np.asarray(u1)
    u2 = np.asarray(u2)

    z1 = stats.norm.ppf(u1)
    z2 = stats.norm.ppf(u2)

    z = np.vstack((z1, z2)).T

    cov_matrix = [[1, rho], [rho, 1]]
    
    mvn_cdf = np.array([stats.multivariate_normal.cdf(z_i, mean=[0, 0], cov=cov_matrix) for z_i in z])
    
    return mvn_cdf

# Example usage:
u1 = [0.5, 0.3, 0.8]
u2 = [0.7, 0.4, 0.9]
rho = 0.6



In [ ]:
def gaussian_copula_pdf_vector(u1, u2, rho):
    '''Cauculate the Gaussian copula density value with given parameter'''
    mean = [0, 0]
    cov = [[1, rho], [rho, 1]]
    inv_u1 = norm.ppf(u1)
    inv_u2 = norm.ppf(u2)

    rv = multivariate_normal(mean, cov)
    inv_uv = np.column_stack((inv_u1, inv_u2))

    pdf_vals = rv.cdf(inv_uv)

    return pdf_vals


def log_copulas_val(u1, u2, rho):
    return np.log(gaussian_copula_pdf_vector(u1, u2, rho))


def conditional_gaussian_copula_vector(u1_vec, u2_vec, rho):
    """
    Compute the conditional copula C(u1 | u2) for vectors of uniform variables using the Gaussian copula.

    Returns:
    numpy.ndarray: Conditional copula values C(u1 | u2) for each pair of (u1, u2).
    """
    u1_vec = np.asarray(u1_vec)
    u2_vec = np.asarray(u2_vec)

    z1_vec = stats.norm.ppf(u1_vec)
    z2_vec = stats.norm.ppf(u2_vec)

    cond_cdf_vec = stats.norm.cdf(
        (z1_vec - rho * z2_vec) / np.sqrt(1 - rho**2))
    if cond_cdf_vec[441]==0:
        print(z1_vec[441],u2_vec[441])
    return cond_cdf_vec

## 2. Discrete Copula
According to the previous section, Sklar's theorem applies under the condition that all marginal distributions are continuous. However, this is not the case for our data, as we used binomial and ZINB models for simulation, both of which have discrete supports (integers). Therefore, we need to use a different approach to handle such discrete data. The method is from Smith, Michael & Khaled, Mohamad. (2011). Estimation of Copula Models With Discrete Margins via Bayesian Data Augmentation. Journal of the American Statistical Association. 107. 10.2139/ssrn.1937983. 

(Note that in this chapter, we focus solely on analyzing two datasets at a time.)

### 2.1 Augmented Distribution
We assume that the two marginal distributions are $X_1$, $X_2$, with $X=(X_1,X_2)$.
Since the marginals for each dataset is discrete, let $a_i = F(x_i^-)$ be the left-hand limit of $F_i$ at $x_i$ and $b_i = F(x_i)$. In our model, we have just $a_i = F(x_i-1)$. Then the probability mass function of $X$ can be written in a closed term:
$$
f(\boldsymbol{x}) = \Delta_{a_1}^{b_1}\Delta_{a_2}^{b_2}C(\boldsymbol{v}),
$$
where $\boldsymbol{v}=(v_1,v_2)$, and the operator here is the difference notation of Nelsen(2006). Expand the right-hand side we get:
\begin{equation}
f(\boldsymbol{x}) = C(b_1,b_2)+C(a_1,a_2)-C(b_1,a_2)-C(a_1,b_2).
\end{equation}
The following code uses the fitted parameters to compute all the values of $a_i$ and $b_i$ for each entry in our data.


In [ ]:
def get_range(data, para):
    N = data.shape[0]
    data_range = pd.DataFrame(index=range(N), columns=data.columns.to_list())
    for col in data.columns:
        para_0 = para[col]
        if not pd.isna(para_0[2]):
            for i in range(N):
                x = data_pro[col][i]
                data_range.loc[i, col] = [x, tl.zinb_cdf(
                    x-1, para_0), tl.zinb_cdf(x, para_0)]
        else:
            for i in range(N):
                x = data_pro[col][i]
                data_range.loc[i, col] = [x, stats.nbinom.cdf(
                    x-1, para_0[0], para_0[1]), stats.nbinom.cdf(x, para_0[0], para_0[1])]
    return data_range


data_range = get_range(data_pro, fitted_para)

Each entry now is a list consists three elements, where the first one is the original entry $x_{ij}$ $(i = 1,2,\dots,13,\ j = 1,2,\dots,N)$ where $N$ is the number of observations. And it is followed by the corresponding values of $a_{ij}$ and $b_{ij}$.

In [ ]:
data_range.head()

In this discrete copula model, we consider a joint distribution of $(X,U)$, with $U=(U_1,U_2)$ where each $U_i$ a uniform distribution on $[0,1]$
- Proposition: If $(X, U)$ has mixed probability density given by:
  $$
  f(\boldsymbol{x},\boldsymbol{u}) = f(\boldsymbol{x}|\boldsymbol{u})f(\boldsymbol{u})=\prod_{j=1}^{m} \mathbb{I}(F_j(x_j^-) \leq u_j < F_j(x_j)) c(\boldsymbol{u}),
\end{equation}
  where $ f(\mathbf{x}|\mathbf{u}) = \prod_{j=1}^{m} f(x_j | u_j)$, then the marginal probability mass function of $X$ is given by equation $(1)$.

In our case, we have $m=2$.
  

Let $\rho$ be the copula parameter, then we can write the mixed PDF as:
$$
f(\boldsymbol{x},\boldsymbol{u}|\rho) = \prod_{i=1}^{n}  c(\boldsymbol{u_i};\rho) \prod_{j=1}^{m=2} \mathbb{I}(a_{ij} \leq u_{ij} < b_{ij}),
$$
where $n$ is the number of observations, $\boldsymbol{u}=(u_1,u_2,\dots,u_n)$ and $\boldsymbol{u_i} = (u_{i1},u_{i2})$

### 2.2 Sampling Schemes
We then use MCMC to estimate the values of $\rho$ and $\boldsymbol{u}$. First we need to know the necessary conditional probabilities.


- **Sample $\boldsymbol{u}$**
    
    Here, we sample $\boldsymbol{u}$ each margin at a time. And we have for $j = 1, 2$:
    $$
    f(\boldsymbol{u}_j|\rho,\boldsymbol{x},\boldsymbol{u}_{k\neq j})\propto \prod_{i=1}^{n} \mathbb{I}(a_{ij} \leq u_{ij} < b_{ij})c_{j|k\neq j}(u_{ij}|u_{ik,k\neq j},\rho)\tag{2}
    $$
    Therefore, according to equation $(2)$, the latents $u_{ij}$ are generated from the conditional densities $c_{j|k\neq j}$ constrained to $[a_{ij} , b_{ij})$, and an iterate for $\boldsymbol{u_j}$ obtained.

    So first, we set the initial values for $\boldsymbol{u}$ with the constrained range for each entry $u_{ij}$:



In [ ]:
def set_initial(data):
    '''Set the initial values for matrix u
    Input:
    Dataframe with each entry be a list of length 3, ordered [x_ij, a_ij, b_ij]

    Return:
    Dataframe with each entry be a list of length 3, ordered [u_ij, a_ij, b_ij]
    '''
    N = data.shape[0]
    data_initial = pd.DataFrame(index=range(N), columns=data.columns.to_list())
    for col in data.columns:
        for i in range(N):
            data_initial.loc[i, col] = [random.uniform(
                data[col][i][1], data[col][i][2])]
            data_initial.loc[i, col] += data[col][i][1:]
    return data_initial


initial_u = set_initial(data_range)
initial_u.head()

For $u$, we know exactly how to sample a new value for $u$ with given $\rho$, so for $u$, we perform MH algorithm here.

In [ ]:

def sample_u(u, ranges, rho):
    '''Get the proposal value for u with given constraint range'''
    u = np.asarray(u)

    # Compute inverse CDF (norm.ppf) for u
    inv2 = norm.ppf(u)
    N = u.shape[0]
    # Extract lower and upper bounds from the ranges matrix
    a = [lis[0] for lis in ranges]
    b = [lis[1] for lis in ranges]

    u_0 = np.random.uniform(size=N)
    cdf_low = conditional_gaussian_copula_vector(a, u, rho)
    cdf_upper = conditional_gaussian_copula_vector(b, u, rho)

    u2 = norm.cdf(norm.ppf((cdf_upper-cdf_low)*u_0+cdf_low)
                  * np.sqrt(1-rho**2)+rho*inv2)
    # last term converges to zero
    print(a[441], b[441], u2[441], ((cdf_upper-cdf_low)*u_0+cdf_low)[441])
    return u2


# sample_u([0.5, 0.4], [[0.6, 0.7], [0.2, 0.8]], 0.5)

In [ ]:
def conditional_gaussian_copula_pdf(u1,u2,rho):
    '''Calculate the value of c(u1|u2;rho)'''
    return norm.pdf(u1,loc = rho*u2,scale = np.sqrt(1-rho^2))

def conditional_pdf_u(u_0,u,range_u,i,rho):
    if u_0>=range_u[0] and u_0 < range_u[1]:
        return conditional_gaussian_copula_pdf(u,u_0,rho)/(conditional_gaussian_copula_vector(range_u[1],u,rho)-conditional_gaussian_copula_vector(range_u[0],u,rho))
    return 0

def proposal_distribution(size=1):
    return np.random.exponential(1, size) 

def rejection_sampling_u(u_0,u, M,rho):       
    u_p = u
    N = u.shape[0]
    col = u.columns
        
    u_condition = [u[col[0]][i][0] for i in range(N)]
    u_0_ranges = [u_0[u_0.columns[0]][i][1:] for i in range(N)]
    #  Draw uniform random variable
    
    for i  in len(u):
        x_proposed = proposal_distribution()
        alpha = np.random.uniform(0, 1)
    #  Accept or reject the sample
        while alpha > conditional_pdf_u(x_proposed,u_condition[i],u_0_ranges[i],i,rho) / (M * np.exp(-u_p)): 
            x_proposed = proposal_distribution()
            alpha = np.random.uniform(0, 1) # p(x) / (M * q(x))
        u_p[i][0] = x_proposed
    return u_p
def get_alpha(u1_p,u1,u2_range,rho):
    term1 = conditional_gaussian_copula_vector(u2_range[1],u1_p,rho)-conditional_gaussian_copula_vector(u2_range[1],u1_p,rho)
    term2 = conditional_gaussian_copula_vector(u2_range[1],u1,rho)-conditional_gaussian_copula_vector(u2_range[1],u1,rho)
    return term1/term2
def mh_prop_u(u1_pack,u2_pack,rho):
    N = u1_pack.shape[0]
    u1= [u1_pack[u1_pack.columns[0]][i][0] for i in range(N)]
    u2_ranges = [u2_pack[u2_pack.columns[0]][i][1:] for i in range(N)]
    u1_p = set_initial(u1)
    u2_p = rejection_sampling_u(u1, 1.0,rho)
    alpha = [get_alpha(u1_p[i],u1[i],u2_ranges[i],rho) for i in range(N)]
    for i,val in enumerate(alpha):
     
        u = np.random.uniform(0, 1)
        if u <= min(1,val):
            u1[i] = u1_p[i]
            u2[i] = u2_p[i]
    return u1,u2

In [ ]:
def update_u(u1_pack, u2_pack, rho):
    '''propose new u_2 given the values form other'''
    N = u1_pack.shape[0]
    col2 = u2_pack.columns
    u1_condition = [u1_pack[u1_pack.columns[0]][i][0] for i in range(N)]
    u2_ranges = [u2_pack[col2[0]][i][1:] for i in range(N)]

    u2 = pd.DataFrame(index=range(N), columns=col2.to_list())

    new_val = sample_u(u1_condition, u2_ranges, rho)
    
    u2[col2[0]] = [[new_val[i], u2_pack[col2[0]][i][1], u2_pack[col2[0]][i][2]]
                   for i in range(N)]
    #print(u2[col2[0]][441])
    return u2

- **Propose $\boldsymbol{\rho}$**
$$
f(\rho|\boldsymbol{u},\boldsymbol{x}) = f(\rho|\boldsymbol{u}) \propto \prod_{i=1}^{n} c(\boldsymbol{u_i};\rho)\pi(\rho)\tag{3}
$$
Here, $\pi(\rho)$ is the prior, and we choose the prior to be the truncated normal distribution between $-1$ and $1$ with standard deviation $\sigma_\rho$. So we have the proprosal algorithm for $\rho$ as following:

In [ ]:
def truncnorm_prop_rho(x, sigma):  # proposal for rho 
    return truncnorm.rvs((-1-x) / sigma, (1-x) / sigma, loc=x, scale=sigma)

def truncnorm_rho_pdf(x, x_p, sigma):
    rv = truncnorm((-1-x) / sigma, (1-x) / sigma, loc=x, scale=sigma)
    return rv.pdf(x_p)

Calculate the log acceptance ratio for the proposal $\rho$.

In [ ]:
def gc_acceptance_ratio(u, para):
    rho, rho_p = para
    u1, u2 = u

    u1 = u1.to_numpy()
    u2 = u2.to_numpy()
    
    u1 = [li[0][0] for li in u1]
    u2 = [li[0][0] for li in u2]

    acc = sum(log_copulas_val(u1, u2, rho_p)) - \
        sum(log_copulas_val(u1, u2, rho))
        
    return acc
# gc_acceptance_ratio(initial_u[['MT-CO1']],initial_u[['MT-CO2']],(0.4,0.5),0.01)

In [ ]:
def nb_marginal(num, r, p, u_raw):

    N, J = num.shape

    rtn = [np.zeros((N, J)), np.zeros((N, J))]

    for j in range(J):
        for n in range(N):

            Ubound = nbinom.cdf(num[n, j], r[j], p[j])
            Lbound = 0
            if num[n, j] > 0:
                Lbound = nbinom.cdf(num[n, j] - 1, r[j], p[j])
            UmL = Ubound - Lbound
            rtn[0][n, j] = norm.ppf(Lbound + UmL * u_raw[n, j])
            rtn[1][n, j] = np.log(UmL)

    return rtn


def apply_ng_marginal(name1, name2, u):
    num = data_pro[[name1, name2]].values
    r = fitted_para.loc[0, [name1, name2]].values
    p = fitted_para.loc[1, [name1, name2]].values
    u_raw = u
    return nb_marginal(num, r, p, u_raw)

In [ ]:
def metropolis_hastings_gc(u1, u2, sigma, n_iter=10000, burn_in=2000, thin=5):

    trace = np.zeros(n_iter)
    trace[0] = 0.5

    acceptance_rate = np.zeros(n_iter)
    rho = trace[0]

    for i in range(1, n_iter):
        rho = trace[i-1]
        rho_p = truncnorm_prop_rho(rho, sigma)


        # Metropolis algorithm to rho
        alpha = gc_acceptance_ratio((u1, u2), (rho, rho_p))
        u = np.log(np.random.uniform(0., 1.))

        if u < alpha:
            trace[i] = rho_p
            acceptance_rate[i-1] = 1

        else:
            trace[i] = rho
    print(u1.columns[0]+' and '+u2.columns[0] +
          " acceptance rate is: %.2f" % acceptance_rate[burn_in:].mean())
    return pd.DataFrame(trace[burn_in::thin])


def metropolis_hastings_para(u1, u2, sigma, n_iter=10000, burn_in=2000, thin=5):
    trace = np.zeros(n_iter)
    trace[0] = [0.5,u1,u2]

    acceptance_rate = np.zeros(n_iter)
    rho = trace[0]
    u1 = trace[1]
    u2 = trace[2]

    for i in range(1, n_iter):
        rho = trace[i-1]
        rho_p = truncnorm_prop_rho(rho, sigma)
        u1_p ,u2_p = mh_prop_u(u1,u2,rho)

        # Metropolis algorithm to u
        alpha_u = update_u((u1, u2),(u1_p,u2_p) rho)
        u = np.log(np.random.uniform(0., 1.))

        # Metropolis algorithm to rho
        alpha_rho = gc_acceptance_ratio((u1, u2), (rho, rho_p))
        u = np.log(np.random.uniform(0., 1.))

        if u < alpha_u:
            trace[i][0] = rho_p
            
        else:
            trace[i] = rho
    print(u1.columns[0]+' and '+u2.columns[0] +
          " acceptance rate is: %.2f" % acceptance_rate[burn_in:].mean())
    return pd.DataFrame(trace[burn_in::thin])

trace_0 = metropolis_hastings_gc(
    initial_u[['MT-CO2']], initial_u[['MT-CO3']], sigma=0.01, n_iter=200, burn_in=0, thin=1)

In [ ]:
plt.plot(trace_0[0])

In [ ]:
cor_o = np.zeros((13,13))

In [ ]:
names = data_range.columns.to_list()
traces = pd.DataFrame(index=names, columns=names)

for i in range(13):
    for j in range(i+1, 13):
        if cor_o[i, j] == 0 or cor_o[i, j] == 0.5:
            tra = metropolis_hastings_gc(initial_u[[names[i]]], initial_u[[
                                         names[j]]], sigma=0.01, n_iter=5000, burn_in=1000, thin=5)
            traces.loc[names[i], names[j]] = [
                tra[:100].mean()-tra[-100:].mean(), tra, tra.mean()]
            cor_o[i, j] = float(tra.mean().iloc[0])
            print(cor_o[i, j])
    print(cor_o)

In [ ]:
df = pd.DataFrame(cor_o, columns=names,index=names)

In [ ]:
cor_o

## 3. Get the Correlatioon

After doing the MCMC, we obatin the convergence values for $u$ and $\rho$. Then, we calciulate the value of kendalls tau using the value of $\rho$, where tau can be written as:
$$
\tau_{i,j}^{F}( \Theta) = \sum_{x_i} \sum_{x_j} f_{i,j}(x_i, x_j; \Theta) \Big[C_{i,j}(F_i, F_j) 
+ C_{i,j}(F_i, \bar{F}_j) + C_{i,j}(\bar{F}_i, F_j) 
+ C_{i,j}(\bar{F}_i, \bar{F}_j) \Big] - 1,

$$

In [ ]:
names = data_pro.columns.to_list()

def kendalls_tau(name1, name2):
    tau = 0

    rho = 0.9# cor_o[pos1, pos2]
    x1 = data_range[name1]
    x2 = data_range[name2]

    for val1 in x1:

        for val2 in x2:
            f = sum(gaussian_copula_pdf_vector([val1[2], val1[1]], [val2[2], val2[1]], rho))-sum(gaussian_copula_pdf_vector([val1[1], val1[2]],
                                                                                                                            [val2[2], val2[1]], rho))
            term2 = sum(gaussian_copula_pdf_vector([val1[2], val1[1], val1[2], val1[1]], [
                val2[2], val2[1], val2[1], val2[2]], rho))
            tau += f*term2
        print(tau)
    tau -= 1
    return tau
# the value for f is too small, guess it is the reason tau is so close to zero with any value of rho.

kendalls_tau('MT-CO1', 'MT-CO2')

### !
 Cannot get a resonable output when calculate kendalls tau.